<p align="center">
  <a href="http://www.openpandemic.io"><img alt="openpandemic" src="https://avatars2.githubusercontent.com/u/63398478?s=100&v=4" width=100 /></a>
  <h3 align="center">Openpandemic - Analytics</h3>
  <p align="center">
  <table style="border-collapse: collapse; border: none;">
<tr>
  <td>
    <img align="center" alt="We love Opensource" src="https://badges.frapsoft.com/os/v1/open-source.svg?v=103" />
  </td>
  <td>
    <a href="https://colab.research.google.com/github/openpandemic/openpandemic-analytics/blob/master/notebooks/covid19/01-Symptoms_exploration.ipynb"><img align="center" alt="Colab" src="https://colab.research.google.com/assets/colab-badge.svg" /></a>
  </td>
  </tr>
  </table>
  </p>
</p>

---

We want to collaborate with the OpenPandemic initiative, a generous gesture in OSS terms to help to stop pandemic diseases.

Please take a look at the [openpandemic-app](https://github.com/OpenPandemic/openpandemic-app) and [openpandemic-back](https://github.com/OpenPandemic/openpandemic-back) repositories.



This notebook is meant to give you a basic entrypoint to explore collected datasets from the application.

# Requirements

*   Google Cloud BigQuery connector (loaded by default in Colab python runtimes)
*   [optional] Data access to the GCP project where data is placed, only if you are not a project member.




In order to get access to GCP run this cell (set the suitable values for variables, name of project is required)

In [ ]:
from google.colab import auth
auth.authenticate_user()

PROJECT='openpandemic-analytics' # SET THE GCP PROJECT NAME
BUCKET='' # SET YOUR GCS BUCKET NAME

import os
ROOT='./'
MODEL_DIR=os.path.join(ROOT,'models')
PACKAGES_DIR=os.path.join(ROOT,'packages')

## Getting started

We have three main options to operate with Bigquery from python kernels

### Magic words (bigquery extensions)

Let's try our first query to BigQuery via magic word. We try to get the total count of evaluations and users

In [ ]:
%load_ext google.cloud.bigquery
%load_ext google.colab.data_table

In [ ]:
%%bigquery --project {PROJECT} --verbose df_total_eval

SELECT
  COUNT(1) as total_evaluation,
  COUNT(DISTINCT person_id) as total_person_count
FROM `openpandemic-analytics.openpandemic_test.data_test_es_v1`

In [ ]:
df_total_eval

### Official python client for Bigquery

We could have done the same queries via bigquery client directly, using native resources or dump results into a pandas dataframe 



In [ ]:
from google.cloud import bigquery as bq

client = bq.Client(project=PROJECT)

dataset_name="openpandemic_test"
table_name="data_test_es_v1"
table_id = f"{PROJECT}.{dataset_name}.{table_name}"

# Query to get total evaluations and person total count
q_summary = f'''
SELECT
  COUNT(1) as total_evaluation,
  COUNT(DISTINCT person_id) as total_person_count
FROM `{table_id}`
'''

query_job = client.query(q_summary)  # API request
rows = query_job.result()            # Waits for query to finish

# Show the summary of items
for row in rows:
  print({k:v for (k,v) in row.items()})

# Dump into pandas dataframe
#df_summary = rows.to_dataframe()
#df_summary.head()                    

### Pandas client for Bigquery

We have a third alternative to get data into a pandas dataframe as well, that is to use [pandas-gbq](https://pypi.org/project/pandas-gbq/) the pandas development to operate with BigQuey.



In [ ]:
import pandas as pd

df = pd.io.gbq.read_gbq(f'''
SELECT
  COUNT(1) as total_evaluation,
  COUNT(DISTINCT person_id) as total_person_count
FROM `{table_id}`
''', project_id=PROJECT, dialect='standard')

df.head()

## Evaluations

We're going to extract the summary of users who have done any evaluation and the latest test result.

In [ ]:
%%bigquery --project {PROJECT} --verbose df_summary_eval

SELECT
 person_id,
 test.id as test_id,
 test.time as test_time,
 test.result as test_result
FROM openpandemic-analytics.openpandemic_test.data_test_es_v1 C
JOIN (
  SELECT
    person_id as person_id1,
    COUNT(*) as eval_count,
    MAX(test.time) as latest_test_time
  FROM `openpandemic-analytics.openpandemic_test.data_test_es_v1`
  GROUP BY person_id
) C1
ON C.person_id = C1.person_id1 AND C.test.time = latest_test_time
ORDER BY PERSON_ID,TEST.TIME DESC

In [ ]:
print('Number of users: %s' % df_summary_eval['person_id'].count())

In [ ]:
df_summary_eval['test_result'].value_counts()

Now, since we have some data about single evaluation per user, let's draw the results

In [ ]:
import plotly.graph_objs as go
from plotly.offline import iplot, plot
import numpy as np

def graph(x, y, title):
    y_sum = sum(y)
    y_text = [ f"{text:.0f}%" for text in np.around((y / y_sum) * 100)]
    data = [go.Bar(
        x=x,
        y=y,
        marker_color='rgba(218, 201, 41, 1)',
        text=y_text,
        textposition = 'auto',
        marker=dict(
            color='rgb(158,202,225)'
        ),
        opacity=0.8)
    ]

    layout = go.Layout(
        title=f'{title}, {y_sum} ',
        paper_bgcolor='rgba(245, 246, 249, 1)',
        plot_bgcolor='rgba(245, 246, 249, 1)',
        showlegend=False,
        xaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            zeroline=True,
            domain=[0.15, 1]
        ),
        yaxis=dict(
            showgrid=True,
            showline=True,
            showticklabels=True,
            zeroline=True,
        )
    )
    return go.Figure(data=data, layout=layout)


In [ ]:
y_counts = df_summary_eval['test_result'].value_counts()

y = (y_counts['symptoms'], y_counts['no-symptoms'],)

x=['Con síntomas compatibles de infección',
   'Sin síntomas compatibles']

title = "Usuarios únicos (identificados o no)"

graph(x, y, title).show()


## Re-evaluations

Now, we're going to find out users who have done more than one evaluation and thier evolution with the symptoms.

We need to know how many evaluations were done by users and wheter symptoms are persistent or not.

In [ ]:
%%bigquery --project {PROJECT} --verbose df_re_eval

SELECT
 person_id,
 test.id as test_id,
 test.time as test_time,
 test.result as test_result,
 C1.eval_count,
 C1.test_no_symptoms_count,
 C1.test_symptoms_count
FROM openpandemic-analytics.openpandemic_test.data_test_es_v1 C
JOIN (
  SELECT
    person_id as person_id1,
    COUNT(*) as eval_count,
    COUNTIF(test.result like "no-symptoms") as test_no_symptoms_count,
    COUNTIF(test.result like "symptoms") as test_symptoms_count,
    MAX(test.time) as latest_test_time
  FROM `openpandemic-analytics.openpandemic_test.data_test_es_v1`
  GROUP BY person_id
  HAVING COUNT(*) > 1
) C1
ON C.person_id = C1.person_id1 AND C.test.time = latest_test_time
ORDER BY PERSON_ID,TEST.TIME DESC

In [ ]:
print('Number of user with more than one evaluation: %s' % df_re_eval['person_id'].count())

In [ ]:
df_re_eval.head(20)

So we have the number of evaluations, negative and positive count of test results and the last result of them then we'd calculate the ratio of users in each situation:

* Users with symptoms (test result were always 'symptoms', eval_count=test_symptoms_count).
* User without symptoms (test result are always 'no-symptoms', eval_count=test_no_symptoms_count).
* Users who now have symptoms (latest test result is 'symptoms' but sometimes before the user had one 'no-symptoms' test result, test_no_symptoms_count>0)
* Users who now have no symptoms (latest test result is 'no-symptoms' but sometimes before the user had one 'symptoms' test result, test_symptoms_count>0)

In [ ]:
users_with_symptoms = df_re_eval[(df_re_eval['test_result'] == "symptoms") & (df_re_eval['test_no_symptoms_count'] == 0)].shape[0]
users_no_symptoms = df_re_eval[(df_re_eval['test_result'] == "no-symptoms") & (df_re_eval['test_symptoms_count'] == 0)].shape[0]
users_changed_symptoms = df_re_eval[(df_re_eval['test_result'] == "symptoms") & (df_re_eval['test_no_symptoms_count'] > 0)].shape[0]
users_changed_no_symptoms = df_re_eval[(df_re_eval['test_result'] == "no-symptoms") & (df_re_eval['test_symptoms_count'] > 0)].shape[0]

In [ ]:
x=['Se mantienen con síntomas',
   'Se mantienen sin síontomas',
   'Cambian su estado a presentar sintomas',
   'Cambian su estado a sin síntomas']

y = np.array([users_with_symptoms, users_no_symptoms, users_changed_symptoms, users_changed_no_symptoms])

re_eval_ratio = np.around((df_re_eval['person_id'].count()/df_total_eval['total_evaluation'])*100)

title = f"Usuarios re-evaluados, {int(re_eval_ratio)}%"

graph(x, y, title).show()


## All in one

We could have done all in few steps, the key is the data understanding and the query we can formed in our minds beforehand.

Let's get the data summary as we need to calculate the results we expect to draw. In this case we're going to classify the type of user regard the the symptoms and evaluations.

In [ ]:
# Query to get the summaty of evaluations
q_summary = f'''
SELECT
 person_id,
 test.id as test_id,
 test.time as test_time,
 test.result as test_result,
 C1.eval_count,
 C1.test_no_symptoms_count,
 C1.test_symptoms_count,
 CASE 
    WHEN (test.RESULT = "symptoms" AND test_no_symptoms_count = 0) THEN "SYMPTOM" 
    WHEN (test.RESULT = "no-symptoms" AND test_symptoms_count = 0) THEN "NO_SYMPTOM"
    WHEN (test.RESULT = "symptoms" AND test_no_symptoms_count > 0) THEN "TO_SYMPTOM"
    WHEN (test.RESULT = "no-symptoms" AND test_symptoms_count > 0) THEN "TO_NO_SYMPTOM"
 END as user_type
FROM `{table_id}` C
JOIN (
  SELECT
    person_id as person_id1,
    COUNT(*) as eval_count,
    COUNTIF(test.result like "no-symptoms") as test_no_symptoms_count,
    COUNTIF(test.result like "symptoms") as test_symptoms_count,
    MAX(test.time) as latest_test_time
  FROM `{table_id}`
  GROUP BY person_id
) C1
ON C.person_id = C1.person_id1 AND C.test.time = latest_test_time
ORDER BY PERSON_ID,TEST.TIME DESC
'''

query_job = client.query(q_summary)  # API request

rows = query_job.result()            # Waits for query to finish

df_summary = rows.to_dataframe()
print(f"Total number of rows: {rows.num_results}\n")
df_summary.head(20)  

In [ ]:
df_summary_unique_eval = df_summary[(df_summary['eval_count'] == 1)]
print('Number of user with just one evaluation: %s\n' % df_summary_unique_eval['person_id'].count())
df_summary_unique_eval.groupby(['user_type'])['person_id'].count()

In [ ]:
df_summary_re_eval = df_summary[(df_summary['eval_count'] > 1)]
print('Number of user with more than one evaluation: %s\n' % df_summary_re_eval['person_id'].count())
df_summary_re_eval.groupby(['user_type'])['person_id'].count()

This query seems to be interesting enought for my coworkers, thus I'm thinking about shere it ... beyond storing it into file in a repository obviously

That's pretty easy, just save it using the bigquery client, for example as a view:


In [ ]:
view_name = "summary_eval_view"
view_id = f"{PROJECT}.openpandemic_test.summary"
view = bq.Table(view_id)
view.view_query = q_summary
view = client.create_table(view)  # API request

print("Successfully created view at {}".format(view.full_table_id))

And now we can use the view to get results: 

In [ ]:
query_job = client.query(f'''
SELECT
  *
FROM `{PROJECT}.openpandemic_test.summary`
''')
rows = query_job.result(max_results=10)
rows.to_dataframe()